In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import MiniBatchKMeans
from scipy.spatial import distance_matrix

In [ ]:
# leitura dos dados
data = pd.read_csv("../data_source/amostra_total.csv", sep=';')
data = data[["INDICE", "LATITUDE", "LONGITUDE", "LOGRADOURO", "NUMERO"]]

In [ ]:
# parâmetros
first_n_clusters = 50
n_of_days = 22

In [ ]:
# clusterização inicial por leiturista
kmeans = MiniBatchKMeans(n_clusters=first_n_clusters, random_state=8081)
data['LEITURISTA'] = kmeans.fit_predict(data[['LATITUDE', 'LONGITUDE']])

In [ ]:
# inicialização das colunas DIA e ROTA
data['DIA'] = -1
data['ROTA'] = -1

In [ ]:
# processamento de cada cluster de leiturista
for leiturista in range(first_n_clusters):
    subcluster_data = data[data['LEITURISTA'] == leiturista]
    if len(subcluster_data) >= n_of_days:
        kmeans_22 = MiniBatchKMeans(n_clusters=n_of_days, random_state=8081)
        subcluster_data['DIA'] = kmeans_22.fit_predict(subcluster_data[['LATITUDE', 'LONGITUDE']])
    else:
        subcluster_data['DIA'] = np.arange(len(subcluster_data))
    
    # atualização do dataframe principal
    data.loc[subcluster_data.index, 'DIA'] = subcluster_data['DIA']

In [ ]:
# criação da coluna ROTA
data['ROTA'] = data.apply(lambda row: row['LEITURISTA'] * n_of_days + row['DIA'], axis=1)

In [ ]:
data.head()

In [ ]:
# Calcular os centróides de cada rota
centroides = data.groupby('ROTA')[['LATITUDE', 'LONGITUDE']].mean().reset_index()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=data, x='LONGITUDE', y='LATITUDE', hue='LEITURISTA', palette='tab20', legend=None)
plt.title('Clusters Iniciais por Leiturista')
plt.show()

In [ ]:
leiturista = 7

plt.figure(figsize=(10, 6))
leiturista_data = data[data['LEITURISTA'] == leiturista]

# rotas com menos de 300 pontos
rotas_menor_300 = leiturista_data['ROTA'].value_counts()[leiturista_data['ROTA'].value_counts() < 300].index

# plotar tudo
sns.scatterplot(data=leiturista_data, x='LONGITUDE', y='LATITUDE', hue='DIA', palette='tab20', legend=None)

# destacar rotas com menos de 300 pontos
menor_300_data = leiturista_data[leiturista_data['ROTA'].isin(rotas_menor_300)]
sns.scatterplot(data=menor_300_data, x='LONGITUDE', y='LATITUDE', hue='DIA', palette='tab20', edgecolor='red', legend=None, linewidth=1)

# centróides ao gráfico
leiturista_centroides = centroides[centroides['ROTA'].isin(leiturista_data['ROTA'])]
plt.scatter(leiturista_centroides['LONGITUDE'], leiturista_centroides['LATITUDE'], s=100, c='black', marker='x')

# destacar os centróides das rotas com menos de 300 pontos
menor_300_centroides = centroides[centroides['ROTA'].isin(rotas_menor_300)]
plt.scatter(menor_300_centroides['LONGITUDE'], menor_300_centroides['LATITUDE'], s=150, c='red', marker='o')

# Conectar centróides de rotas vizinhas com menos de 300 pontos
dist_matrix = distance_matrix(menor_300_centroides[['LATITUDE', 'LONGITUDE']], menor_300_centroides[['LATITUDE', 'LONGITUDE']])
np.fill_diagonal(dist_matrix, np.inf)  # Preencher a diagonal com valores infinitos para evitar distância zero consigo mesmo

for i in range(len(menor_300_centroides)):
    nearest_neighbor_idx = np.argmin(dist_matrix[i])
    plt.plot([menor_300_centroides.iloc[i]['LONGITUDE'], menor_300_centroides.iloc[nearest_neighbor_idx]['LONGITUDE']],
             [menor_300_centroides.iloc[i]['LATITUDE'], menor_300_centroides.iloc[nearest_neighbor_idx]['LATITUDE']], 'k--')

plt.title(f'Rotas para o Leiturista {leiturista}')
plt.show()

In [ ]:
leiturista = 7

plt.figure(figsize=(10, 6))
leiturista_data = data[data['LEITURISTA'] == leiturista]

# rotas com menos de 300 pontos
rotas_menor_300 = leiturista_data['ROTA'].value_counts()[leiturista_data['ROTA'].value_counts() < 300].index

# plotar tudo
sns.scatterplot(data=leiturista_data, x='LONGITUDE', y='LATITUDE', hue='DIA', palette='tab20', legend=None)

# destacar rotas com menos de 300 pontos
menor_300_data = leiturista_data[leiturista_data['ROTA'].isin(rotas_menor_300)]
sns.scatterplot(data=menor_300_data, x='LONGITUDE', y='LATITUDE', hue='DIA', palette='tab20', edgecolor='red', legend=None, linewidth=1)

# centróides ao gráfico
leiturista_centroides = centroides[centroides['ROTA'].isin(leiturista_data['ROTA'])]
plt.scatter(leiturista_centroides['LONGITUDE'], leiturista_centroides['LATITUDE'], s=100, c='black', marker='x')

# destacar os centróides das rotas com menos de 300 pontos
menor_300_centroides = centroides[centroides['ROTA'].isin(rotas_menor_300)]
plt.scatter(menor_300_centroides['LONGITUDE'], menor_300_centroides['LATITUDE'], s=150, c='red', marker='o')

# Conectar centróides de rotas vizinhas com menos de 300 pontos
dist_matrix = distance_matrix(menor_300_centroides[['LATITUDE', 'LONGITUDE']], menor_300_centroides[['LATITUDE', 'LONGITUDE']])
np.fill_diagonal(dist_matrix, np.inf)  # Preencher a diagonal com valores infinitos para evitar distância zero consigo mesmo

for i in range(len(menor_300_centroides)):
    nearest_neighbor_idx = np.argmin(dist_matrix[i])
    rota_atual = menor_300_centroides.iloc[i]['ROTA']
    rota_vizinha = menor_300_centroides.iloc[nearest_neighbor_idx]['ROTA']
    
    # Verificar se a soma dos pontos dos dois clusters é menor ou igual a 500
    soma_pontos = (leiturista_data['ROTA'].value_counts()[rota_atual] + 
                   leiturista_data['ROTA'].value_counts()[rota_vizinha])
    
    if soma_pontos <= 500:
        plt.plot([menor_300_centroides.iloc[i]['LONGITUDE'], menor_300_centroides.iloc[nearest_neighbor_idx]['LONGITUDE']],
                 [menor_300_centroides.iloc[i]['LATITUDE'], menor_300_centroides.iloc[nearest_neighbor_idx]['LATITUDE']], 'k--')

plt.title(f'Rotas para o Leiturista {leiturista}')
plt.show()

In [ ]:
leiturista = 2

plt.figure(figsize=(10, 6))
leiturista_data = data[data['LEITURISTA'] == leiturista]

# rotas com menos de 300 pontos
rotas_menor_300 = leiturista_data['ROTA'].value_counts()[leiturista_data['ROTA'].value_counts() < 300].index

# plotar tudo
sns.scatterplot(data=leiturista_data, x='LONGITUDE', y='LATITUDE', hue='DIA', palette='tab20', legend=None)

# destacar rotas com menos de 300 pontos
menor_300_data = leiturista_data[leiturista_data['ROTA'].isin(rotas_menor_300)]
sns.scatterplot(data=menor_300_data, x='LONGITUDE', y='LATITUDE', hue='DIA', palette='tab20', edgecolor='red', legend=None, linewidth=1)

# centróides ao gráfico
leiturista_centroides = centroides[centroides['ROTA'].isin(leiturista_data['ROTA'])]
plt.scatter(leiturista_centroides['LONGITUDE'], leiturista_centroides['LATITUDE'], s=100, c='black', marker='x')

# destacar os centróides das rotas com menos de 300 pontos
menor_300_centroides = centroides[centroides['ROTA'].isin(rotas_menor_300)]
plt.scatter(menor_300_centroides['LONGITUDE'], menor_300_centroides['LATITUDE'], s=150, c='red', marker='o')

# Conectar centróides de rotas vizinhas com menos de 300 pontos e unir clusters
dist_matrix = distance_matrix(menor_300_centroides[['LATITUDE', 'LONGITUDE']], menor_300_centroides[['LATITUDE', 'LONGITUDE']])
np.fill_diagonal(dist_matrix, np.inf)  # Preencher a diagonal com valores infinitos para evitar distância zero consigo mesmo

for i in range(len(menor_300_centroides)):
    nearest_neighbor_idx = np.argmin(dist_matrix[i])
    rota_atual = menor_300_centroides.iloc[i]['ROTA']
    rota_vizinha = menor_300_centroides.iloc[nearest_neighbor_idx]['ROTA']
    
    # Verificar se a soma dos pontos dos dois clusters é menor ou igual a 500
    soma_pontos = (leiturista_data['ROTA'].value_counts()[rota_atual] + 
                   leiturista_data['ROTA'].value_counts()[rota_vizinha])
    
    if soma_pontos <= 500:
        plt.plot([menor_300_centroides.iloc[i]['LONGITUDE'], menor_300_centroides.iloc[nearest_neighbor_idx]['LONGITUDE']],
                 [menor_300_centroides.iloc[i]['LATITUDE'], menor_300_centroides.iloc[nearest_neighbor_idx]['LATITUDE']], 'k--')
        
        # Unir os clusters no dataframe
        data.loc[data['ROTA'] == rota_vizinha, 'ROTA'] = rota_atual

# Recalcular os centróides após a união dos clusters
centroides = data.groupby('ROTA')[['LATITUDE', 'LONGITUDE']].mean().reset_index()

plt.title(f'Rotas para o Leiturista {leiturista}')
plt.show()

# Plotar mapa atualizado
plt.figure(figsize=(10, 6))
sns.scatterplot(data=data[data['LEITURISTA'] == leiturista], x='LONGITUDE', y='LATITUDE', hue='ROTA', palette='tab20', legend=None)
plt.title(f'Clusters Atualizados para o Leiturista {leiturista}')
plt.show()

# Printar o número de diferentes rotas para o leiturista
num_rotas = len(data[data['LEITURISTA'] == leiturista]['ROTA'].unique())
print(f'Número de diferentes rotas para o leiturista {leiturista}: {num_rotas}')